## Install necessary libraries

In [1]:
!pip install beautifulsoup4 requests sqlalchemy selenium


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from sqlalchemy import create_engine
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time


## Next we test for response 200

In [3]:
url='https://www.aliexpress.com/w/wholesale-laptops.html?g=y&SearchText=laptops'

r = requests.get(url)

print(r.status_code)

200


# After donwloading and extracting the google chrome driver in the file path of our folder, we then configure selenium chrome driver options

In [4]:
# Configure Selenium ChromeDriver options
options = Options()
options.headless = True
service = Service(executable_path=r'chromedriver-win64\chromedriver.exe')

# Initialize the webdriver
driver = webdriver.Chrome(service=service,options=options)

#Define the URL
url= 'https://www.aliexpress.com/w/wholesale-laptops.html?g=y&SearchText=laptops'

# Use Selenium to open the page


#Wait for the Dynamic content to load
time.sleep(10)

# Get the page source and close the browser
page_source= driver.page_source
driver.quit()

#  Extraction Layer

In [43]:
# Configure Selenium ChromeDriver options
options = Options()
options.headless = True
service = Service(executable_path=r'chromedriver-win64\chromedriver.exe')

# Initialize the webdriver
driver = webdriver.Chrome(service=service,options=options)

# List to store extracted data
product_names=[]
prices=[]
store_link=[]
store_names=[]
shipping_prices=[]
extra_discounts=[]
item_sold=[]
original_price=[]
shipping_free_status=[]
sales_prices=[]

# Assuming you know the total number of pages
total_pages=1

#so lets use selenium to stabilize the page and use beautiful soup to extra_discounts

for  page_number in range(1,total_pages + 1):
    url='https://www.aliexpress.com/w/wholesale-laptops.html?g=y&SearchText=laptops'
    driver.get(url)
    time.sleep(10)

    # Now use beautifulsoup to parse the loaded page_source
    # Get the page source 
    #page_source= driver.page_source
    soup= BeautifulSoup(driver.page_source,'html.parser')

    #Next lets adjust the class selectors based on the current website structure
    laptops = soup.find_all('div',class_ ='multi--modalContext--1Hxqhwi')

    for laptop in laptops:
        #product name
        product_name=laptop.find('div',class_ ='multi--title--G7dOCj3').text

        #price
        
        price=laptop.find('div', class_ ='multi--price-sale--U-S0jtj').text.replace('￡','').replace(',','')

        #store name
        store_name=laptop.find('a', class_ ='cards--storeLink--XkKUQFS').text

        #store_link(becuase we are not sure if all the stores have a link, we use the try and except method just incase its not there)
        try:
            store_links= laptop.find('a', class_ ='cards--storeLink--XkKUQFS')['href']
        except (TypeError,KeyError):
            store_link=''

        #shipping_price(here we will also use the try and except clause)
        try:
            shipping_price=laptop.find('span',class_ ='tag--text--1BSEXVh tag--textStyle--3dc7wLU multi--serviceStyle--1Z6RxQ4').text
        except AttributeError:
            shipping_price=''

        # extra discount
        try:
            extra_discount=laptop.find('span', class_ ='tag--text--1BSEXVh tag--textStyle--3dc7wLU multi--superStyle--1jUmObG').text
        except AttributeError:
            extra_discount=''

        # Item sold
        try:
            item_sold_text=laptop.find('span', class_ ='multi--trade--Ktbl2jB').text
            item_sold_value=item_sold_text.split()[0] if item_sold_text else ''
        except AttributeError:
            item_sold_value=''

        # Append data to list

        product_names.append( product_name)
        prices.append(price)
        store_link.append(store_links) 
        store_names.append( store_name)     
        shipping_prices.append(shipping_price)
        extra_discounts.append(extra_discount)
        item_sold.append(item_sold_value)

        # Now lets go ahead to extract the remaining additional information
        #original_prices
        former_price=laptop.find('div',class_ ='multi--price-original--1zEQqOK').text.replace('￡','').replace(',','')
        original_price.append(former_price)
        #original_price.append(former_price[0].text if former_price else '')
        
        #shipping_free_status
        free_shipping=laptop.find_all('div',class_ = 'multi--serviceContainer--3vRdzWN')
        ship_free=free_shipping[0].text.strip() if free_shipping else ''
        shipping_free_status.append(ship_free)

        #sales_prices=[]
        

# Once all the pages information are extracted,quit the driver
driver.quit()


# CREATE A DATAFRAME

# Next lets create a dataframe for our data collected above
# but first lets put them into a dictionary
data={
    'Product_Name': product_names,
    'Prices': prices,
    'Store_Link':store_link,
    'Store_Name': store_name,
    'Shipping_Prices':shipping_prices,
    'Extra_Discounts':extra_discounts,
    'Item_Sold':item_sold,
    'Original_Prices':original_price,
    'Shipping_free_status':shipping_free_status,
}

# Next we put this dictionary into a dataframe
df=pd.DataFrame(data)



In [44]:
#display our top records
display(df.head())

,Product_Name,Prices,Store_Link,Store_Name,Shipping_Prices,Extra_Discounts,Item_Sold,Original_Prices,Shipping_free_status
0,"14.1"" Ultra Slim Laptop 16GB RAM 2TB SSD Intel...",147.14,//www.aliexpress.com/store/1102996501,5G Original Global Tablet Store,Free shipping,Extra 2% off with coins,500+,,Free shipping
1,"Portable 15.6"" Laptop 32GB Ram DDR4 2TB SSD Wi...",196.66,//www.aliexpress.com/store/1103309564,5G Original Global Tablet Store,Free shipping,Extra 2% off with coins,600+,,Free shipping
2,15.6 Inch Laptop 32GB Ram 2TB SSD Windows 11 N...,161.67,//www.aliexpress.com/store/1103515369,5G Original Global Tablet Store,Free shipping,,25,175.29,Free shipping
3,Adreamer LeoBook13 Laptop 8GB RAM 1TB SSD Comp...,116.94,//www.aliexpress.com/store/1100347101,5G Original Global Tablet Store,Free shipping,Extra 5% off with coins,354,,Free shipping
4,"Ultra Slim Laptop 14.1"" 16GB RAM 2TB SSD Intel...",149.72,//www.aliexpress.com/store/1103309564,5G Original Global Tablet Store,Free shipping,Extra 2% off with coins,900+,,Free shipping
